In [1]:
from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Span, Slope, ColorBar, Range1d, Circle, HoverTool, MultiSelect
from bokeh.transform import linear_cmap, factor_cmap, factor_mark
from bokeh.palettes import Spectral6, Plasma, Viridis, Colorblind
import json
import numpy as np
import os
import pandas as pd

output_notebook()
output_file("FinalProject_ES.html", title='Final Project - ES')

Loading BokehJS ...

In [2]:
en_path = os.path.join("..","surface_energies.csv")
surfaces = pd.read_csv(en_path)
#surfaces

In [3]:
en_path = os.path.join("..","doi_10.5061_dryad.f2n6f__v1")
json_file = os.path.join(en_path,"surfaces.json")
with open(json_file,'r') as read_file:
        data = json.load(read_file)
#data

In [4]:
df_ong = pd.DataFrame(data[:], columns= list(data[0].keys()))

In [5]:
df = df_ong.join(surfaces.set_index('Atom'), on='pretty_formula')

In [6]:
atoms = df["pretty_formula"]

pairs = []
diffs = []
cores = []
shells = []
core_surf = []
shell_surf = []
core_num = []
shell_num = []
interf = []

for i in range(len(atoms)):
    for j in range(len(atoms)):
        if i == j:
            continue
            
        pairs.append(str(atoms[i] + "-" + atoms[j])) #A-B pairing
        gamma_b = df["weighted_surface_energy"][j]
        gamma_a = df["weighted_surface_energy"][i]
        diffs.append(gamma_b - gamma_a) # B-A; <0 is core-shell
        cores.append(atoms[i])
        shells.append(atoms[j])
        core_surf.append(gamma_a)
        shell_surf.append(gamma_b)
        core_num.append(df['Atomic Number'][i])
        shell_num.append(df['Atomic Number'][j])
        sigma = ( np.random.rand() * (gamma_b - gamma_a) + gamma_a) / 10.0
        interf.append(sigma)
        
data = {'Pair': pairs,
        'Difference': diffs,
        'Core': cores,
        'Shell' : shells,
        'Core Surface Energy': core_surf,
        'Shell Surface Energy': shell_surf,
        'Core Atomic Number': core_num,
        'Shell Atomic Number': shell_num,
        'Interfacial Energy' : interf
        }

df = pd.DataFrame(data, columns = ['Core','Core Atomic Number','Shell','Shell Atomic Number',
                                   'Pair','Core Surface Energy','Shell Surface Energy', 'Difference','Interfacial Energy'])

df["Shapes"] = np.random.choice([-1,0,1],size=len(df["Core"])) 

#df

In [7]:
CS_COLOR = 'red'
DNE_COLOR = 'white'
HALF_COLOR = 'blue'

num_to_shapes = dict([(-1,'NP not observed'),
                    (0, 'Core-Shell'),
                    (1, 'Heterodimer')])
SHAPES = ['Not observed','Core-Shell','Heterodimer']

colormap = dict([(-1,DNE_COLOR),
                    (0, CS_COLOR),
                    (1, HALF_COLOR)])
df['colors'] = [colormap[x] for x in df['Shapes']]
df['Shapes'] = [num_to_shapes[x] for x in df['Shapes']]

In [8]:
START_CORE = 40
START_SHELL = 41

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(core_num=df["Core Atomic Number"],
                                    core_surf=df["Core Surface Energy"],
                                    shell_num=df["Shell Atomic Number"],
                                    shell_surf=df["Shell Surface Energy"],
                                    diff = df["Difference"],
                                    sigma=df['Interfacial Energy'],
                                    shape=df["Shapes"],
                                    core=df["Core"],
                                    shell=df["Shell"],
                                    pair=df["Pair"],
                                     color=df['colors']))

sub_df = df.loc[df['Core Atomic Number'] == START_CORE]
f_source = ColumnDataSource(data=dict(core_num=sub_df["Core Atomic Number"],
                                    core_surf=sub_df["Core Surface Energy"],
                                    shell_num=sub_df["Shell Atomic Number"],
                                    shell_surf=sub_df["Shell Surface Energy"],
                                    diff = sub_df["Difference"],
                                    sigma=sub_df['Interfacial Energy'],
                                    shape=sub_df["Shapes"],
                                    core=sub_df["Core"],
                                    shell=sub_df["Shell"],
                                    pair=sub_df["Pair"],
                                     color=sub_df['colors']))



TOOLS = "wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create y=x line
slope = Slope(gradient=1, y_intercept=0, line_color='black', line_dash='dashed', line_width=3)

#create horizontal line at y=location
vline1 = Span(location=2, dimension='height', line_color='gray', line_dash='dashed', line_width=2)
vline2 = Span(location=10, dimension='height', line_color='gray', line_dash='dashed', line_width=2)
vline3 = Span(location=18, dimension='height', line_color='gray', line_dash='dashed', line_width=2)
vline4 = Span(location=36, dimension='height', line_color='gray', line_dash='dashed', line_width=2)
vline5 = Span(location=54, dimension='height', line_color='gray', line_dash='dashed', line_width=2)
vline6 = Span(location=86, dimension='height', line_color='gray', line_dash='dashed', line_width=2)

In [9]:
# create another new plot and add a renderer
main = figure(width=650, height=700, title="Core-Shell Surface Energy Comparison")
main.min_border_left = 150
main.min_border_top = 20
main.toolbar_location = None

# plot all the data
main.circle('core_surf', 'shell_surf',source=source,fill_alpha=1,color='lightgray',line_color='lightgray')

main.circle('core_surf', 'shell_surf',
              color='color',
            legend='shape',
              fill_alpha = 0.8,
              source=f_source,
             line_color='color')

main.yaxis.axis_label = 'Shell Weighted Surface Energy, γ (J/m²)'
main.xaxis.axis_label = 'Core Weighted Surface Energy, γ (J/m²)'
main.add_layout(slope)
main.legend.location = "top_left"
#main.legend.click_policy="hide" #for interactive legend

main.x_range=Range1d(-0.2, 4.0) # current data range is 0 to 3.5
main.y_range=Range1d(-0.2, 4.5)

main.add_tools(HoverTool(
    tooltips=[
        ( "Core-Shell",  "@pair" ),
        ("(x,y)", "($x, $y)")
    ]
))

In [10]:
# create a new plot and add a renderer
diff = figure(width=450, height=350,title='Selected Cores - Selected Shells')
diff.min_border_left = 100
diff.min_border_bottom = 75
diff.min_border_top = 20
  
lc = diff.circle('shell_num', 'diff', source=f_source, color='color',name='left')

l_xmin, l_xmax = df["Shell Atomic Number"].min(), df["Shell Atomic Number"].max()
l_ymin, l_ymax = df["Difference"].min(), df["Difference"].max()

lmp1 = diff.multi_polygons(xs=[[[[l_xmin*0.9, l_xmin*0.9, l_xmax*1.1, l_xmax*1.1]]]],
                 ys=[[[[0, l_ymax*1.1, l_ymax*1.1, 0]]]], color=HALF_COLOR,alpha=0.2,line_width=0) # top

lmp2= diff.multi_polygons(xs=[[[[l_xmin*0.9, l_xmin*0.9, l_xmax*1.1, l_xmax*1.1]]]],
                 ys=[[[[0, l_ymin*0.9, l_ymin*0.9, 0]]]], color=CS_COLOR,alpha=0.2,line_width=0) # bottom

diff.yaxis.axis_label = 'Difference in Weighted Surface Energies, Δγ (J/m²)'
diff.xaxis.axis_label = 'Shell Atomic #'

diff.add_tools(HoverTool(
    tooltips=[
        ( "Core-Shell",  "@pair" ),
        ("(x,y)", "($x, $y)")
    ],
    names=['left'],
    renderers=[lc]
))

diff.add_layout(vline1)
diff.add_layout(vline2)
diff.add_layout(vline3)
diff.add_layout(vline4)
diff.add_layout(vline5)
diff.add_layout(vline6)

In [11]:
# create another new plot and add a renderer
interf = figure(width=450, height=350, title=None)
r_xmin, r_xmax = df['Interfacial Energy'].min(), df['Interfacial Energy'].max()
r_ymin, r_ymax = df['Difference'].min(), df['Difference'].max() # one is - of the other
r_min = np.min([r_xmin,r_ymin])
r_max = np.max([r_xmax,r_ymax])
interf.circle('sigma', 'diff', source=f_source, color='color',name='right')

interf.add_layout(slope)

interf.yaxis.axis_label = 'Difference in Surface Energies, Δγ (J/m²)'
interf.xaxis.axis_label = 'Interfacial Energy, σ (J/m²)'

interf.add_tools(HoverTool(
    tooltips=[
        ( "A-B",  "@pair" ),
        ("(x,y)", "($x, $y)")
    ],
    names=['right']
))

In [12]:
OPTIONS = [(str(x), y) for x, y in zip(df["Shell Atomic Number"].unique(), df["Shell"].unique())]
OPTIONS = (sorted(OPTIONS, key = lambda x: x[1]))  
values_list = [row[0] for row in OPTIONS]

In [13]:
# put the subplots in a gridplot
grid1 = gridplot( [[main]],merge_tools=True,toolbar_options=dict(logo=None))
grid2 = gridplot( [[diff],[interf]], merge_tools=True,toolbar_options=dict(logo=None))

core_slider = Slider(start=df["Core Atomic Number"].min(), end=df["Core Atomic Number"].max(),
                     value=START_CORE, step=1, title="Core Atomic #")
shell_slider = Slider(start=df["Shell Atomic Number"].min(), end=df["Shell Atomic Number"].max(),
                      value=START_SHELL, step=1, title="Shell Atomic #")

multi_select = MultiSelect(value=values_list, options=OPTIONS,title="Select Shell(s): ",height=200)

sliderCallback = CustomJS(args=dict(source=source, core=core_slider, shell=multi_select, f_source=f_source), code="""
    var data = source.data;
           
    var gamma_a = [];
    var gamma_b = [];
    var dGamma = [];
    var shell_n = [];
    var sig = [];
    var core_n = [];
    var shapes = [];
    var c = [];
    var s = [];
    var p = [];
    var col = [];
    
    for (var i = 0; i < data.shell_num.length; i++) {
        for (var j = 0; j < shell.value.length; j++) {
            if (data.core_num[i] == core.value && data.shell_num[i] == shell.value[j]) {
                gamma_a.push(data.core_surf[i]);
                gamma_b.push(data.shell_surf[i]);
                dGamma.push(data.diff[i]);
                shell_n.push(data.shell_num[i]);
                core_n.push(data.core_num[i]);
                sig.push(data.sigma[i]);
                shapes.push(data.shape[i]);
                c.push(data.core[i]);
                s.push(data.shell[i]);
                p.push(data.pair[i]);
                col.push(data.color[i]);
            }
        }
    }
    
    f_source.data.core_surf = gamma_a
    f_source.data.shell_surf = gamma_b
    f_source.data.diff = dGamma
    f_source.data.shell_num = shell_n
    f_source.data.core_num = core_n
    f_source.data.sigma = sig
    f_source.data.shape = shapes
    f_source.data.core = c
    f_source.data.shell = s
    f_source.data.pair = p
    f_source.data.color = col
    
    f_source.change.emit();
""")

msCallback = CustomJS(args=dict(source=source, core=core_slider, shell=multi_select, f_source=f_source),code="""
    var data = source.data;
    var gamma_a = [];
    var gamma_b = [];
    var dGamma = [];
    var shell_n = [];
    var sig = [];
    var core_n = [];
    var shapes = [];
    var c = [];
    var s = [];
    var p = [];
    var col = [];
    
    for (var i = 0; i < data.shell_num.length; i++) {
        for (var j = 0; j < this.value.length; j++) {
            if (data.core_num[i] == core.value && data.shell_num[i] == this.value[j]) {
                gamma_a.push(data.core_surf[i]);
                gamma_b.push(data.shell_surf[i]);
                dGamma.push(data.diff[i]);
                shell_n.push(data.shell_num[i]);
                core_n.push(data.core_num[i]);
                sig.push(data.sigma[i]);
                shapes.push(data.shape[i]);
                c.push(data.core[i]);
                s.push(data.shell[i]);
                p.push(data.pair[i]);
                col.push(data.color[i]);
            }
        }
    }
    
    f_source.data.core_surf = gamma_a
    f_source.data.shell_surf = gamma_b
    f_source.data.diff = dGamma
    f_source.data.shell_num = shell_n
    f_source.data.core_num = core_n
    f_source.data.sigma = sig
    f_source.data.shape = shapes
    f_source.data.core = c
    f_source.data.shell = s
    f_source.data.pair = p
    f_source.data.color = col
    f_source.change.emit();
""")


multi_select.js_on_change("value", msCallback)
core_slider.js_on_change('value', sliderCallback)

layout = row( grid1, grid2, column(gridplot( [[None]]), core_slider, multi_select)  )

show(layout)